#Step 0 mount your own drive with shared files



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1 import all lib

In [ ]:
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, utils
import pandas as pd
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
# %matplotlib notebook
from math import hypot
from itertools import combinations
from sklearn.model_selection import KFold
import gc
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
from tensorflow.keras import regularizers
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# from sklearn.metrics import root_mean_squared_error
import pickle
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Step 2 : Run this Block : 
## Prepare Data : contain 3 functions 

In [ ]:
def create_dataset(X, y):
    features = []
    targets = []
    
    for i in range(0, len(X)): 
        data = [[i] for i in X[i]] # 序列数据  
        label = [y[i]] # 标签数据
        
        # 保存到features和labels
        features.append(data)
        targets.append(label)
    
    # 返回
    return np.array(features), np.array(targets)

In [ ]:
# split data 
#x_train, x_test, y_train, y_test

def split_dataset(x, y, train_ratio=0.8):

    x_len = len(x) # 特征数据集X的样本数量
    train_data_len = int(x_len * train_ratio) # 训练集的样本数量
    
    x_train = x[:train_data_len] # 训练集
    y_train = y[:train_data_len] # 训练标签集
    
    x_test = x[train_data_len:] # 测试集
    y_test = y[train_data_len:] # 测试集标签集
    
    # 返回值
    return x_train, x_test, y_train, y_test

# Step 3: Run this Block 
## Model build base on Delete NaN File 
### Need to change file path to your own drive file path

In [ ]:
seed = 2

In [ ]:
filenum = 1
gsize = 50
shuffle = True

In [ ]:
dataset_x = []
dataset_y = []
import random

with open('/content/drive/MyDrive/ColabNotebooks/{size}mm_file/outfile{fnum}/trainingfile_{size}mm.txt'.format(size = gsize, fnum = filenum), 'r') as f:
    lines = f.readlines()
    if shuffle:
      random.Random(seed).shuffle(lines)
    else:
      pass
    # print(lines[10])
    for line in lines:
        line = line.strip("\n")
        dataset_x.append(line.split("|")[0].split(","))
        dataset_y.append(line.split("|")[1])
        

In [ ]:
len(dataset_x)

16

In [ ]:
dataset_x
lable = [float(y) for y in dataset_y]
input_x = []
for grp in dataset_x:
  input_x.append([float(z) for z in grp])

In [ ]:
input_x,lable = create_dataset(input_x, lable)
x_train, x_test, y_train, y_test = split_dataset(input_x, lable, train_ratio=0.80)

In [ ]:
nsample,nx,ny = x_train.shape
x_train_2d = x_train.reshape(nsample, nx*ny)

nsamplet,nxt,nyt = x_test.shape
x_test_2d = x_test.reshape(nsamplet, nxt*nyt)

In [ ]:
''' try to get best performed params'''

# C = [0.1, 0.2, 0.5, 0.8, 0.9, 1, 2, 5, 10]
# kernel = 'rbf'
# gamma = [0.001, 0.01, 0.1, 0.2, 0.5, 0.8]
# epsilon = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8]
# # 参数字典
# params_dict = {
#     'C': C,
#     'gamma': gamma,
#     'epsilon': epsilon
# }

# # 创建SVR实例
# svr = SVR()

# # 网格参数搜索
# gsCV = GridSearchCV(
#     estimator=svr,
#     param_grid=params_dict,
#     n_jobs=2,
#     scoring='r2',
#     cv=6
# )



# # 输出参数信息
# print("最佳度量值:", gsCV.best_score_)
# print("最佳参数:", gsCV.best_params_)
# print("最佳模型:", gsCV.best_estimator_)



In [ ]:
# build model and get result
svr=SVR(C = 10, kernel = 'rbf', epsilon= 0.1, gamma= 0.001)
svr=svr.fit(x_train_2d, y_train.ravel())



svr_predict=svr.predict(x_test_2d)


mae = mean_absolute_error(y_test, svr_predict)
mse = mean_squared_error(y_test, svr_predict)
rmse = mean_squared_error(y_test, svr_predict, squared = False)
r2 = r2_score(y_test, svr_predict)



# print(svr_predict)
print("RMSE：", rmse)
print("MAE：", mae)
print("MSE：", mse)
print("R^2：", r2)



RMSE： 1.1680736395536346
MAE： 0.9250195618530016
MSE： 1.3643960274200742
R^2： 0.23727039096133418


In [ ]:
# save model
pkl_filename = '/content/drive/MyDrive/ColabNotebooks/{size}mm_file/outfile{fnum}/Model_svm.pickle'.format(
      size = gsize, fnum = filenum)

with open(pkl_filename, 'wb') as file:
    pickle.dump(svr, file)

In [ ]:
# # 设置超参数
# C = [0.1, 0.2, 0.5, 0.8, 0.9, 1, 2, 5, 10]
# kernel = 'rbf'
# gamma = [0.001, 0.01, 0.1, 0.2, 0.5, 0.8]
# epsilon = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8]
# # 参数字典
# params_dict = {
#     'C': C,
#     'gamma': gamma,
#     'epsilon': epsilon
# }

# # 创建SVR实例
# svr = SVR()

# # 网格参数搜索
# gsCV = GridSearchCV(
#     estimator=svr,
#     param_grid=params_dict,
#     n_jobs=2,
#     scoring='r2',
#     cv=6
# )

# # 输出参数信息
# print("最佳度量值:", gsCV.best_score_)
# print("最佳参数:", gsCV.best_params_)
# print("最佳模型:", gsCV.best_estimator_)